In [ ]:
pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [ ]:
%pip

In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
img = cv2.imread(os.path.join("data", "Evaluation_Set", "Validation", "284.png"))
img.shape

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# tf.data.Dataset??

In [ ]:
import numpy as py
from matplotlib import pyplot as plt

In [ ]:
tf.keras.utils.image_dataset_from_directory??

In [ ]:
training_data = tf.keras.utils.image_dataset_from_directory('data/Training_Set/Training/')
eval_data = tf.keras.utils.image_dataset_from_directory('data/Evaluation_Set/Validation/')
testing_data = tf.keras.utils.image_dataset_from_directory('data/Test_Set/Test/')

In [ ]:
data_iterator_training = training_data.as_numpy_iterator()
data_iterator_eval = eval_data.as_numpy_iterator()
data_iterator_testing = testing_data.as_numpy_iterator()

In [ ]:
batch_training = data_iterator_training.next()
batch_eval = data_iterator_eval.next()
batch_testing = data_iterator_testing.next()

In [ ]:
batch_training[0].shape

In [ ]:
#0 normal?
#1 not normal
batch_training[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch_training[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch_training[1][idx])

In [ ]:
batch_training[0].max()

In [ ]:
training_data = training_data.map(lambda x,y: (x/255,y))
eval_data = eval_data.map(lambda x,y: (x/255,y))
testing_data = testing_data.map(lambda x,y: (x/255,y))

In [ ]:
data_iterator_training_scaled = training_data.as_numpy_iterator()
data_iterator_eval_scaled = eval_data.as_numpy_iterator()
data_iterator_testing_scaled = testing_data.as_numpy_iterator()

In [ ]:
batch_training_scaled = data_iterator_training_scaled.next()
batch_eval_scaled = data_iterator_eval_scaled.next()
batch_testing_scaled = data_iterator_testing_scaled.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch_training_scaled[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch_training_scaled[1][idx])

In [ ]:
len(training_data)

In [ ]:
len(testing_data)

In [ ]:
len(eval_data)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(training_data, epochs=20, validation_data=eval_data, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re =Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in testing_data.as_numpy_iterator():
    X ,y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
img = cv2.imread('data/Test_Set/Test/Normal/245.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
import numpy as np
np.expand_dims(resize,0)

In [ ]:
yhat = model.predict(np.expand_dims(resize/255,0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print(f'Predicted image is abnormal')
else:
    print(f'Predicted image is normal')